In [1]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### 1. 대상지 주소 입력 > 법정동코드 추출

In [2]:
from urllib.parse import quote
with open('roadapikey.txt') as file:
    road_key = f'{file.read()}'

In [4]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote(input())}&resultType=json'
url = f"{road_url}?{params1}&{params2}"
result = requests.get(url).json()

BJD_code=result['results']['juso'][0]['admCd'] 
BJD_code

'2647010200'

### 2. 법정동코드 > 대상지 기후대, 토양성분 추출

In [11]:
addrr = pd.read_csv('강원도.csv', sep=',')  #전국.csv만들기
addrr = pd.DataFrame(addrr)
addrr = addrr[['EMNDN_CD','CLZN_CD']]
addrr.columns = ['읍면동코드','기후대코드']
addrr['읍면동코드'] = int(BJD_code[:-2])
climate_code = addrr['기후대코드'].mode()[0]
climate_code  

# 1) 온대북부, 2)온대중부, 3)온대남부 4) 난대

# if climate_code == 1.0:
#     print('온대북부')
# elif climate_code == 2.0:
#     print('온대중부')
# elif climate_code == 3.0:
#     print('온대남부')
# elif climate_code == 4.0:
#     print('난대')

2.0

In [13]:
with open('gapikey.txt') as file:
    gkey = file.read()

s_url = f'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey={gkey}&Page_Size=10&Page_No=1&BJD_Code={BJD_code}'
s_result = requests.get(s_url)
s_contents = s_result.text
s_soup = BeautifulSoup(s_contents, 'html.parser')

acid = []
vldpha = []
vldsia = []
om = []
posifert_mg = []
posifert_k = []
posifert_ca = []
selc= []

lis = [acid, vldpha, vldsia, om, posifert_mg, posifert_k, posifert_ca, selc]
cons = ['acid', 'vldpha', 'vldsia', 'om', 'posifert_mg', 'posifert_k', 'posifert_ca', 'selc']

for li in lis:
    for con in cons:
        for a in s_soup.find_all(con):
            li.append(float(a.get_text()))

ground_con = pd.DataFrame({'산도':acid, '유효인산':vldpha, '유효규산':vldsia, '유기물':om, '마그네슘':posifert_mg, '칼륨':posifert_k , '칼슘':posifert_ca, '전기전도도':selc})
ground_con

,산도,유효인산,유효규산,유기물,마그네슘,칼륨,칼슘,전기전도도
0,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00
1,6.10,6.10,6.10,6.10,6.10,6.10,6.10,6.10
2,6.20,6.20,6.20,6.20,6.20,6.20,6.20,6.20
3,5.30,5.30,5.30,5.30,5.30,5.30,5.30,5.30
4,5.30,5.30,5.30,5.30,5.30,5.30,5.30,5.30
...,...,...,...,...,...,...,...,...
75,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.40
76,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15
77,1.40,1.40,1.40,1.40,1.40,1.40,1.40,1.40
78,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10
